In [1]:
!curl -sSL "https://julialang-s3.julialang.org/bin/linux/x64/1.2/julia-1.2.0-linux-x86_64.tar.gz" -o julia.tar.gz
!tar -xzf julia.tar.gz -C /usr --strip-components 1
!rm -rf julia.tar.gz*
!julia -e 'using Pkg; pkg"add IJulia; add CuArrays; add Flux; precompile"'

   Cloning default registries into `~/.julia`
   Cloning registry from "https://github.com/JuliaRegistries/General.git"
     Added registry `General` to `~/.julia/registries/General`
 Resolving package versions...
 Installed VersionParsing ── v1.2.0
 Installed Parsers ───────── v1.0.2
 Installed Conda ─────────── v1.4.1
 Installed MbedTLS ───────── v0.7.0
 Installed IJulia ────────── v1.21.2
 Installed JSON ──────────── v0.21.0
 Installed ZMQ ───────────── v1.1.0
 Installed BinaryProvider ── v0.5.9
 Installed SoftGlobalScope ─ v1.0.10
  Updating `~/.julia/environments/v1.2/Project.toml`
  [7073ff75] + IJulia v1.21.2
  Updating `~/.julia/environments/v1.2/Manifest.toml`
  [b99e7846] + BinaryProvider v0.5.9
  [8f4d0f93] + Conda v1.4.1
  [7073ff75] + IJulia v1.21.2
  [682c06a0] + JSON v0.21.0
  [739be429] + MbedTLS v0.7.0
  [69de0a69] + Parsers v1.0.2
  [b85f4697] + SoftGlobalScope v1.0.10
  [81def892] + VersionParsing v1.2.0
  [c2297ded] + ZMQ v1.1.0
  [2a0f44e3] + Base64 
  [ade2ca70] +

In [2]:
1

1

In [3]:
1+1

2

In [1]:
[1,2,3]

3-element Array{Int64,1}:
 1
 2
 3

In [2]:
import Pkg
Pkg.add("Ipopt")
Pkg.add("JuMP")
Pkg.add("Interpolations")
Pkg.add("Dierckx")

using Ipopt
using JuMP
using Interpolations
using Dierckx

┌ Info: Precompiling JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1186
┌ Info: Recompiling stale cache file C:\Users\Оля\.julia\compiled\v1.1\Interpolations\VpKVx.ji for Interpolations [a98d9a8b-a2ab-59e6-89dd-64a1c18fca59]
└ @ Base loading.jl:1184


In [3]:
tau = 60 #seconds
# N = 24*3600 / tau
# N = round(N)
N = 1440
# N = 60
time_t = zeros(1,N)
# time_t[i] = (i-1)*60/3600 for i=1:N
# prices_t = price(time_t)
COP = 3
C_out = 1005 # J/(kg K)
C_in = 1005 # J/(kg K)
m_air = 340 # kg
m_wall = 11520/2 #kg
C_wall = 1500 # J/(kg K)
pa1 = tau/(m_air*C_in)
h = 5 # m
b = 10 # m
k = 0.183 
pa2 = h*b*k
alpha_int = 8.7 # W/ m K
S = 60 # m^2
pa3 = 0.5 * alpha_int * h * (S / b + b)
M_met = 120 # W
T_vent = 21 # ^oC
a_inf = 0.02 / 3600
pa4 = a_inf * m_air * C_out
M_light = 20 # W
N_light = 15
W_light = M_light * N_light
L_evap = 333000 # W/kg
M_moist = 0.000021 # kg/s
pa5 = M_met - L_evap * M_moist
pw1 = tau/(m_wall*C_wall)
pn1 = tau/m_air
pn2 = a_inf * m_air
k_CO2 = 0.038 * 4 # g / s # medium work
ppm_k_CO2 = k_CO2 / 1000 / (S * h) * 1000000 # mass to kg divide by volume and to ppm
T_output = 17 # outer air temperature after TES

charge_start = 1
charge_stop = 420
discharge_start = 541
discharge_stop = 1080

# charge_start = round(0*3600/tau) + 1
# charge_stop = round(7*3600/tau)
# discharge_start = round(9*3600/tau + 1)
# discharge_stop = round(18*3600/tau)

1080

In [4]:
N = 300

300

In [5]:
m2 = Model(Ipopt.Optimizer)

@variable(m2, P_c[1:N], base_name="compressor", lower_bound=0, upper_bound=3)
@variable(m2, P_f[1:N], base_name="fan", lower_bound=0, upper_bound=4)

# @variable(m2, W_hvac[1:1N], lower_bound=0, upper_bound=10000)
# @variable(m2, W_dis[1:N], lower_bound=0, upper_bound=10000)
# @variable(m2, W_ch[1:N], lower_bound=0, upper_bound=10000)

# @variable(m2, Q[1:N], lower_bound=0, upper_bound=100)
# @variable(m2, E[1:N], lower_bound=0, upper_bound=10000000)
@variable(m2, T_r[1:N], lower_bound=0, upper_bound=50)
@variable(m2, T_w[1:N], lower_bound=0, upper_bound=50)
@variable(m2, n_c[1:N], lower_bound=0, upper_bound=2000)

@expression(m2, expr_price[i = 1:N],LinearInterpolation([0, 0.999, 1, 1.999, 2, 2.999, 3, 3.999, 4, 4.999, 5, 5.999, 6, 6.999, 7, 7.999, 8, 8.999, 9, 9.999, 10, 10.999, 11, 11.999, 12, 12.999, 13, 13.999, 14, 14.999, 15, 15.999, 16, 16.999, 17, 17.999, 18, 18.999, 19, 19.999, 20, 20.999, 21, 21.999, 22, 22.999, 23, 23.999, 24, 24.999], [4.41704, 4.41704, 4.25404, 4.25404, 4.14436, 4.14436, 4.05468, 4.05468, 4.01459, 4.01459, 4.02631, 4.02631, 4.16135, 4.16135, 4.24251, 4.24251, 4.49976, 4.49976, 4.59363, 4.59363, 4.77394, 4.77394, 4.80014, 4.80014, 4.78849, 4.78849, 4.75757, 4.75757, 4.77814, 4.77814, 4.79783, 4.79783, 4.7674, 4.7674, 4.73724, 4.73724, 4.72584, 4.72584, 4.73167, 4.73167, 4.73894, 4.73894, 4.7544, 4.7544, 4.75318, 4.75318, 4.61478, 4.61478, 4.41704, 4.41704],extrapolation_bc=Flat())(i/60))

@expression(m2, expr_obj_init[i in 1:N], 10 * P_f[i] + 500 * P_c[i])

@expression(m2, expr_obj, sum(expr_obj_init[j] * expr_price[j] for j in 1:N))

@objective(m2, Min, expr_obj)

# @constraint(m2,con2, W_hvac - COP * 500 * P_c + W_dis - W_ch == 0)
# @constraint(m2,con2[i in 1:N], W_hvac[i] - COP * 500 * P_c[i] == 0)

# @constraint(m2,con3[i in 1:N], Q[i] - 0.55 * P_f[i] * 10 / 100 == 0)

# @constraint(m2,con4, E[1] == 0)
@constraint(m2,con5, T_r[1] == 19)
@constraint(m2,con6, T_w[1] == 19)
@constraint(m2,con7, n_c[1] == 400)

@constraint(m2,con8[i in 2:N], T_r[i] >= 18)
@constraint(m2,con9[i in 2:N], T_r[i] <= 24)
@constraint(m2,con10[i in 2:N], n_c[i] <= 800)

@expression(m2, expr_out[i = 1:N], Spline1D([0, 3, 6 , 9, 12, 15, 18, 21, 25], [19, 17, 18, 20, 25, 23, 21, 19, 18])(i/60))

@expression(m2, expr_occup[i = 1:N], LinearInterpolation([0, 8.9, 9, 12, 12.1, 15, 15.1, 18, 18.1], [0, 0,   20,20, 30,   30, 10,    10,  0],extrapolation_bc=Flat())(i/60))

@NLconstraint(m2,nl1[i in 1:(N-1)], T_r[i+1] - (T_r[i] + pa1*((pa2 + pa4) * (expr_out[i] - T_r[i]) + pa3 * (T_w[i] - T_r[i]) + pa5 * expr_occup[i] - COP * 500 * P_c[i] + C_out * T_vent * 0.55 * P_f[i] * 10 / 100 - C_in * T_r[i] * 0.55 * P_f[i] * 10 / 100 + W_light)) == 0)

@constraint(m2,con11[i in 1:(N-1)], T_w[i+1] - (T_w[i] + pw1 * pa2 * (T_r[i] - T_w[i])) == 0)

@NLconstraint(m2,nl2[i in 1:(N-1)], n_c[i+1] - (n_c[i] + pn1 * (0.55 * P_f[i] * 10 / 100 + pn2) * (300 - n_c[i]) + k_CO2 * expr_occup[i]) == 0)

optimize!(m2)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     3292
Number of nonzeros in inequality constraint Jacobian.:      897
Number of nonzeros in Lagrangian Hessian.............:     1794

Total number of variables............................:     1500
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1500
                     variables with only upper bounds:        0
Total number of equ

In [6]:
println(JuMP.value.(P_c))
println(JuMP.value.(P_f))
println(JuMP.objective_value(m2))
println(JuMP.termination_status(m2))

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [7]:
N = 1440

1440

In [8]:
# USUAL MPC


m3 = Model(Ipopt.Optimizer)

@variable(m3, P_c[1:N], base_name="compressor", lower_bound=0, upper_bound=3)
@variable(m3, P_f[1:N], base_name="fan", lower_bound=0, upper_bound=4)

# @variable(m3, W_hvac[1:1N], lower_bound=0, upper_bound=10000)
# @variable(m3, W_dis[1:N], lower_bound=0, upper_bound=10000)
# @variable(m3, W_ch[1:N], lower_bound=0, upper_bound=10000)

# @variable(m3, Q[1:N], lower_bound=0, upper_bound=100)
# @variable(m3, E[1:N], lower_bound=0, upper_bound=10000000)
@variable(m3, T_r[1:N], lower_bound=0, upper_bound=50)
@variable(m3, T_w[1:N], lower_bound=0, upper_bound=50)
@variable(m3, n_c[1:N], lower_bound=0, upper_bound=2000)

@expression(m3, expr_price[i = 1:N],LinearInterpolation([0, 0.999, 1, 1.999, 2, 2.999, 3, 3.999, 4, 4.999, 5, 5.999, 6, 6.999, 7, 7.999, 8, 8.999, 9, 9.999, 10, 10.999, 11, 11.999, 12, 12.999, 13, 13.999, 14, 14.999, 15, 15.999, 16, 16.999, 17, 17.999, 18, 18.999, 19, 19.999, 20, 20.999, 21, 21.999, 22, 22.999, 23, 23.999, 24, 24.999], [4.41704, 4.41704, 4.25404, 4.25404, 4.14436, 4.14436, 4.05468, 4.05468, 4.01459, 4.01459, 4.02631, 4.02631, 4.16135, 4.16135, 4.24251, 4.24251, 4.49976, 4.49976, 4.59363, 4.59363, 4.77394, 4.77394, 4.80014, 4.80014, 4.78849, 4.78849, 4.75757, 4.75757, 4.77814, 4.77814, 4.79783, 4.79783, 4.7674, 4.7674, 4.73724, 4.73724, 4.72584, 4.72584, 4.73167, 4.73167, 4.73894, 4.73894, 4.7544, 4.7544, 4.75318, 4.75318, 4.61478, 4.61478, 4.41704, 4.41704],extrapolation_bc=Flat())(i/60))

@expression(m3, expr_obj_init[i in 1:N], 10 * P_f[i] + 500 * P_c[i])

@expression(m3, expr_obj, sum(expr_obj_init[j] * expr_price[j] for j in 1:N))

@objective(m3, Min, expr_obj)

# @constraint(m3,con2, W_hvac - COP * 500 * P_c + W_dis - W_ch == 0)
# @constraint(m3,con2[i in 1:N], W_hvac[i] - COP * 500 * P_c[i] == 0)

# @constraint(m3,con3[i in 1:N], Q[i] - 0.55 * P_f[i] * 10 / 100 == 0)

# @constraint(m3,con4, E[1] == 0)
@constraint(m3,con5, T_r[1] == 19)
@constraint(m3,con6, T_w[1] == 19)
@constraint(m3,con7, n_c[1] == 400)

@constraint(m3,con8[i in 2:N], T_r[i] >= 18)
@constraint(m3,con9[i in 2:N], T_r[i] <= 24)
@constraint(m3,con10[i in 2:N], n_c[i] <= 800)

@expression(m3, expr_out[i = 1:N], Spline1D([0, 3, 6 , 9, 12, 15, 18, 21, 25], [19, 17, 18, 20, 25, 23, 21, 19, 18])(i/60))

@expression(m3, expr_occup[i = 1:N], LinearInterpolation([0, 8.9, 9, 12, 12.1, 15, 15.1, 18, 18.1], [0, 0,   20,20, 30,   30, 10,    10,  0],extrapolation_bc=Flat())(i/60))

@NLconstraint(m3,nl1[i in 1:(N-1)], T_r[i+1] - (T_r[i] + pa1*((pa2 + pa4) * (expr_out[i] - T_r[i]) + pa3 * (T_w[i] - T_r[i]) + pa5 * expr_occup[i] - COP * 500 * P_c[i] + C_out * T_vent * 0.55 * P_f[i] * 10 / 100 - C_in * T_r[i] * 0.55 * P_f[i] * 10 / 100 + W_light)) == 0)

@constraint(m3,con11[i in 1:(N-1)], T_w[i+1] - (T_w[i] + pw1 * pa2 * (T_r[i] - T_w[i])) == 0)

@NLconstraint(m3,nl2[i in 1:(N-1)], n_c[i+1] - (n_c[i] + pn1 * (0.55 * P_f[i] * 10 / 100 + pn2) * (300 - n_c[i]) + k_CO2 * expr_occup[i]) == 0)

optimize!(m3)

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    15832
Number of nonzeros in inequality constraint Jacobian.:     4317
Number of nonzeros in Lagrangian Hessian.............:     8634

Total number of variables............................:     7200
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     7200
                     variables with only upper bounds:        0
Total number of equality constraints.................:     4320
Total number of inequality constraints...............:     4317
        inequality constraints with only lower bounds:     1439
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:     2878

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

In [9]:
println(JuMP.value.(P_c))
println(JuMP.value.(P_f))
println(JuMP.objective_value(m3))
println(JuMP.termination_status(m3))

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [ ]:
# TES USING


m4 = Model(Ipopt.Optimizer)

@variable(m4, P_c[1:N], base_name="compressor", lower_bound=0, upper_bound=3)
@variable(m4, P_f[1:N], base_name="fan", lower_bound=0, upper_bound=4)

@variable(m4, W_hvac[1:N], lower_bound=0, upper_bound=10000)
@variable(m4, W_dis[1:N], lower_bound=0, upper_bound=10000)
@variable(m4, W_ch[1:N], lower_bound=0, upper_bound=10000)

# @variable(m4, Q[1:N], lower_bound=0, upper_bound=100)
@variable(m4, E[1:N], lower_bound=0, upper_bound=10000000)
@variable(m4, T_r[1:N], lower_bound=0, upper_bound=50)
@variable(m4, T_w[1:N], lower_bound=0, upper_bound=50)
@variable(m4, n_c[1:N], lower_bound=0, upper_bound=2000)

@expression(m4, expr_price[i in 1:N],LinearInterpolation([0, 0.999, 1, 1.999, 2, 2.999, 3, 3.999, 4, 4.999, 5, 5.999, 6, 6.999, 7, 7.999, 8, 8.999, 9, 9.999, 10, 10.999, 11, 11.999, 12, 12.999, 13, 13.999, 14, 14.999, 15, 15.999, 16, 16.999, 17, 17.999, 18, 18.999, 19, 19.999, 20, 20.999, 21, 21.999, 22, 22.999, 23, 23.999, 24, 24.999], [4.41704, 4.41704, 4.25404, 4.25404, 4.14436, 4.14436, 4.05468, 4.05468, 4.01459, 4.01459, 4.02631, 4.02631, 4.16135, 4.16135, 4.24251, 4.24251, 4.49976, 4.49976, 4.59363, 4.59363, 4.77394, 4.77394, 4.80014, 4.80014, 4.78849, 4.78849, 4.75757, 4.75757, 4.77814, 4.77814, 4.79783, 4.79783, 4.7674, 4.7674, 4.73724, 4.73724, 4.72584, 4.72584, 4.73167, 4.73167, 4.73894, 4.73894, 4.7544, 4.7544, 4.75318, 4.75318, 4.61478, 4.61478, 4.41704, 4.41704],extrapolation_bc=Flat())(i/60))

@expression(m4, expr_obj_init[i in 1:N], 10 * P_f[i] + 500 * P_c[i])

@expression(m4, expr_obj, sum(expr_obj_init[j] * expr_price[j] for j in 1:N))

@objective(m4, Min, expr_obj)

@constraint(m4,con2[i in 1:N], W_hvac[i] - COP * 500 * P_c[i] + W_dis[i] - W_ch[i] == 0)
# @constraint(m4,con2[i in 1:N], W_hvac[i] - COP * 500 * P_c[i] == 0)

# @constraint(m4,con3[i in 1:N], Q[i] - 0.55 * P_f[i] * 10 / 100 == 0)

@constraint(m4,con4, E[1] == 0)
@constraint(m4,con5, T_r[1] == 19.8)
@constraint(m4,con6, T_w[1] == 19)
@constraint(m4,con7, n_c[1] == 400)

@constraint(m4,con8[i in 2:N], T_r[i] >= 18)
@constraint(m4,con9[i in 2:N], T_r[i] <= 24)
@constraint(m4,con10[i in 2:N], n_c[i] <= 800)

@expression(m4, expr_out[i = 1:N], Spline1D([0, 3, 6 , 9, 12, 15, 18, 21, 25], [19, 17, 18, 20, 25, 23, 21, 19, 18])(i/60))

@expression(m4, expr_occup[i = 1:N], LinearInterpolation([0, 8.9, 9, 12, 12.1, 15, 15.1, 18, 18.1], [0, 0,   20,20, 30,   30, 10,    10,  0],extrapolation_bc=Flat())(i/60))


@constraint(m4,con11[i in 1:(N-1)], E[i+1] - (E[i] - tau * (W_dis[i] - W_ch[i])) == 0)

@constraint(m4,con12[i in charge_start:charge_stop], W_ch[i] - W_hvac[i] == 0)

@constraint(m4,con13[i in (charge_stop + 1):N], W_ch[i] == 0)

@constraint(m4,con14[i in 1:(discharge_start - 1)], W_dis[i] == 0)

@constraint(m4,con15[i in discharge_start:discharge_stop], W_dis[i] - (C_out * 0.55 * P_f[i] * 10 / 100 * (expr_out[i] - T_output)) == 0)

@constraint(m4,con16[i in (discharge_stop + 1):N], W_dis[i] == 0)

@constraint(m4,con18[i in discharge_start:discharge_stop], P_c[i] == 0)

@NLconstraint(m4,nl1[i in 1:(N-1)], T_r[i+1] - (T_r[i] + pa1*((pa2 + pa4) * (expr_out[i] - T_r[i]) + pa3 * (T_w[i] - T_r[i]) + pa5 * expr_occup[i] - W_hvac[i] + C_out * T_vent * 0.55 * P_f[i] * 10 / 100 - C_in * T_r[i] * 0.55 * P_f[i] * 10 / 100 + W_light)) == 0)

@constraint(m4,con17[i in 1:(N-1)], T_w[i+1] - (T_w[i] + pw1 * pa2 * (T_r[i] - T_w[i])) == 0)

@NLconstraint(m4,nl2[i in 1:(N-1)], n_c[i+1] - (n_c[i] + pn1 * (0.55 * P_f[i] * 10 / 100 + pn2) * (300 - n_c[i]) + k_CO2 * expr_occup[i]) == 0)

optimize!(m4)

In [0]:
println(JuMP.value.(P_c))
println(JuMP.value.(P_f))
println(JuMP.objective_value(m4))
println(JuMP.termination_status(m4))

In [2]:
Pkg.add("Juniper")
using Juniper

UndefVarError: ignored

In [1]:
optimizer = Juniper.Optimizer
nl_solver = optimizer_with_attributes(Ipopt.Optimizer, "print_level"=>0)

m10 = Model(optimizer_with_attributes(optimizer, "nl_solver"=>nl_solver))

@variable(m10, P_c[1:1440], base_name="compressor", lower_bound=0, upper_bound=3, Int)
@variable(m10, P_f[1:1440], base_name="fan", lower_bound=0, upper_bound=4, Int)

# @variable(m10, W_hvac[1:1N], lower_bound=0, upper_bound=10000)
# @variable(m10, W_dis[1:N], lower_bound=0, upper_bound=10000)
# @variable(m10, W_ch[1:N], lower_bound=0, upper_bound=10000)

# @variable(m10, Q[1:N], lower_bound=0, upper_bound=100)
# @variable(m10, E[1:N], lower_bound=0, upper_bound=10000000)
@variable(m10, T_r[1:N], lower_bound=0, upper_bound=50)
@variable(m10, T_w[1:N], lower_bound=0, upper_bound=50)
@variable(m10, n_c[1:N], lower_bound=0, upper_bound=2000)

@expression(m10, expr_price[i = 1:N],LinearInterpolation([0, 0.999, 1, 1.999, 2, 2.999, 3, 3.999, 4, 4.999, 5, 5.999, 6, 6.999, 7, 7.999, 8, 8.999, 9, 9.999, 10, 10.999, 11, 11.999, 12, 12.999, 13, 13.999, 14, 14.999, 15, 15.999, 16, 16.999, 17, 17.999, 18, 18.999, 19, 19.999, 20, 20.999, 21, 21.999, 22, 22.999, 23, 23.999, 24, 24.999], [4.41704, 4.41704, 4.25404, 4.25404, 4.14436, 4.14436, 4.05468, 4.05468, 4.01459, 4.01459, 4.02631, 4.02631, 4.16135, 4.16135, 4.24251, 4.24251, 4.49976, 4.49976, 4.59363, 4.59363, 4.77394, 4.77394, 4.80014, 4.80014, 4.78849, 4.78849, 4.75757, 4.75757, 4.77814, 4.77814, 4.79783, 4.79783, 4.7674, 4.7674, 4.73724, 4.73724, 4.72584, 4.72584, 4.73167, 4.73167, 4.73894, 4.73894, 4.7544, 4.7544, 4.75318, 4.75318, 4.61478, 4.61478, 4.41704, 4.41704],extrapolation_bc=Flat())(i/60))

@expression(m10, expr_obj_init[i in 1:N], 10 * P_f[i] + 500 * P_c[i])

@expression(m10, expr_obj, sum(expr_obj_init[j] * expr_price[j] for j in 1:N))

@objective(m10, Min, expr_obj)

# @constraint(m10,con2, W_hvac - COP * 500 * P_c + W_dis - W_ch == 0)
# @constraint(m10,con2[i in 1:N], W_hvac[i] - COP * 500 * P_c[i] == 0)

# @constraint(m10,con3[i in 1:N], Q[i] - 0.55 * P_f[i] * 10 / 100 == 0)

# @constraint(m10,con4, E[1] == 0)
@constraint(m10,con5, T_r[1] == 19)
@constraint(m10,con6, T_w[1] == 19)
@constraint(m10,con7, n_c[1] == 400)

@constraint(m10,con8[i in 2:N], T_r[i] >= 18)
@constraint(m10,con9[i in 2:N], T_r[i] <= 24)
@constraint(m10,con10[i in 2:N], n_c[i] <= 800)

@expression(m10, expr_out[i = 1:N], Spline1D([0, 3, 6 , 9, 12, 15, 18, 21, 25], [19, 17, 18, 20, 25, 23, 21, 19, 18])(i/60))

@expression(m10, expr_occup[i = 1:N], LinearInterpolation([0, 8.9, 9, 12, 12.1, 15, 15.1, 18, 18.1], [0, 0,   20,20, 30,   30, 10,    10,  0],extrapolation_bc=Flat())(i/60))

@NLconstraint(m10,nl1[i in 1:(N-1)], T_r[i+1] - (T_r[i] + pa1*((pa2 + pa4) * (expr_out[i] - T_r[i]) + pa3 * (T_w[i] - T_r[i]) + pa5 * expr_occup[i] - COP * 500 * P_c[i] + C_out * T_vent * 0.55 * P_f[i] * 10 / 100 - C_in * T_r[i] * 0.55 * P_f[i] * 10 / 100 + W_light)) == 0)

@constraint(m10,con11[i in 1:(N-1)], T_w[i+1] - (T_w[i] + pw1 * pa2 * (T_r[i] - T_w[i])) == 0)

@NLconstraint(m10,nl2[i in 1:(N-1)], n_c[i+1] - (n_c[i] + pn1 * (0.55 * P_f[i] * 10 / 100 + pn2) * (300 - n_c[i]) + k_CO2 * expr_occup[i]) == 0)

optimize!(m10)

UndefVarError: ignored

In [0]:
println(JuMP.value.(P_c))
println(JuMP.value.(P_f))
println(JuMP.objective_value(m10))
println(JuMP.termination_status(m10))